# Importing data from any external database as a table dataset in azure ML

In [1]:
from azure.ai.ml.entities import DataImport
from azure.ai.ml.data_transfer import Database
from azure.ai.ml import MLClient

In [2]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import InteractiveBrowserCredential
credential = InteractiveBrowserCredential(tenant_id="13a86542-2185-4187-8e07-7512f5525c55")

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="84a5808b-5549-459a-98f2-f102e84fa1bb",
    resource_group_name="EYMAY",
    workspace_name="aml-workspace"
)

Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [3]:
from azure.ai.ml.entities import WorkspaceConnection,UsernamePasswordConfiguration
target = "Server=tcp:anshu-server.database.windows.net,1433;Initial Catalog=anshu-azure-sql;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30"

name = "emp_data_azure_sql"
wps_connection = WorkspaceConnection(name = name,
                                     type="azure_sql_db",
                                     target=target,
                                     credentials=UsernamePasswordConfiguration(username="anshu",password="Qwerty@12345"))

ml_client.connections.create_or_update(workspace_connection=wps_connection)

Class WorkspaceConnection: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


WorkspaceConnection({'type': 'azure_sql_db', 'target': 'Server=tcp:anshu-server.database.windows.net,1433;Initial Catalog=anshu-azure-sql;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30', 'credentials': {'type': 'username_password'}, 'metadata': None, 'name': 'emp_data_azure_sql', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspaces/aml-workspace/connections/emp_data_azure_sql', 'Resource__source_path': None, 'base_path': 'd:\\AI\\MLOps\\EYMAY23', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x00000213C9C25DB0>})

## Working with keyvault to load secrets

In [3]:
from azure.identity import InteractiveBrowserCredential
credential = InteractiveBrowserCredential(tenant_id="13a86542-2185-4187-8e07-7512f5525c55")

In [4]:
import os
from azure.keyvault.secrets import SecretClient


keyVaultName = "amlanshukeyvault5076bfa1"
KVUri = f"https://{keyVaultName}.vault.azure.net"

client = SecretClient(vault_url=KVUri, credential=credential)

In [5]:
secretName = input("Input a name for your secret > ")
secretValue = input("Input a value for your secret > ")

print(f"Creating a secret in {keyVaultName} called '{secretName}' with the value '{secretValue}' ...")

client.set_secret(secretName, secretValue)

print(" done.")


Creating a secret in amlanshukeyvault5076bfa1 called 'DBpassword' with the value 'Qwerty@12345' ...
 done.


In [6]:
print(f"Retrieving your secret from {keyVaultName}.")

retrieved_secret = client.get_secret(secretName)

print(f"Your secret is '{retrieved_secret.value}'.")
print(f"Deleting your secret from {keyVaultName} ...")

poller = client.begin_delete_secret(secretName)
deleted_secret = poller.result()

print(" done.")

Retrieving your secret from amlanshukeyvault5076bfa1.
Your secret is 'Qwerty@12345'.
Deleting your secret from amlanshukeyvault5076bfa1 ...
 done.


## Working with Azure SQL Databases

In [24]:
import pyodbc
server = 'myserver-anshu.database.windows.net'
database = 'myDB'
username = 'anshu'
password = 'Qwerty@12345'   
driver= '{ODBC Driver 18 for SQL Server}'


In [1]:

with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT * from dbo.emp;")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]) + " " + str(row[2]))
            row = cursor.fetchone()

1 John Doe
2 Jane Doe
3 Anshu Pandey
4 Kelly Peter
5 Macon Dary
